In [1]:
!git clone https://github.com/yuliya1324/data_for_workshop.git

Cloning into 'data_for_workshop'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 85 (delta 15), reused 16 (delta 5), pack-reused 46
Unpacking objects: 100% (85/85), done.


In [2]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 41.9 MB/s 
     |████████████████████████████████| 182 kB 53.7 MB/s 


In [8]:
import pandas as pd
from tqdm.auto import tqdm
import re
import os
from statistics import mean

In [14]:
from transformers import BertTokenizer, BertModel, BertConfig

model_version = 'sberbank-ai/ruBert-base'
model = BertModel.from_pretrained(model_version, output_attentions=True, num_attention_heads=12)
tokenizer = BertTokenizer.from_pretrained(model_version)

Downloading:   0%|          | 0.00/590 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of the model checkpoint at sberbank-ai/ruBert-base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

In [7]:
with open('./data_for_workshop/semantic_roles_data.csv', encoding='utf-8', errors='ignore') as csvfile:
    role_df = pd.read_csv(csvfile, sep=',')

role_df.head(3)

,sentence,role,idx_target,raw_target,idx_head,raw_head
0,«Школа злословия» учит прикусить язык,agent,1-6,Школа,18-22,учит
1,Сохранится ли градус дискуссии в новом сезоне?,posessor,14-20,градус,0-10,Сохранится
2,Великолепная «Школа злословия» вернулась в эфи...,agent,14-19,Школа,31-40,вернулась


In [9]:
os.chdir('data_for_workshop/xmshikunova')

In [10]:
from attention_exp import *

In [11]:
role_df = role_df[role_df['idx_head'] != 'oops']

In [12]:
role_df = role_df[role_df['role']=='posessor']

In [15]:
weighted_df_list = get_matrix(role_df, tokenizer, model, tqdm)

  0%|          | 0/54 [00:00<?, ?it/s]

In [16]:
weighted_df_list.keys()

dict_keys(['vmean_amean', 'vmean_amax', 'vmean_ast', 'vmax_amean', 'vmax_amax', 'vmax_ast', 'vst_amean', 'vst_amax', 'vst_ast'])

In [17]:
df = weighted_df_list['vmean_amean']

In [18]:
get_mean_df(df[df['role']=='posessor'], mean)


,h_0,h_1,h_2,h_3,h_4,h_5,h_6,h_7,h_8,h_9,h_10,h_11,layer
0,0.055721,0.053050,0.050091,0.039800,0.066042,0.019285,0.062405,0.035073,0.054217,0.114023,0.039881,0.213733,0
1,0.016930,0.024100,0.031809,0.028330,0.136186,0.129118,0.098763,0.001840,0.027489,0.046495,0.067231,0.033573,1
2,0.055020,0.043092,0.015271,0.043413,0.023356,0.051259,0.047960,0.002012,0.084151,0.114903,0.013558,0.048117,2
3,0.052461,0.029705,0.017135,0.011093,0.024543,0.031982,0.016721,0.049362,0.047472,0.122538,0.036593,0.002959,3
4,0.073290,0.049742,0.000540,0.095812,0.030029,0.249823,0.044999,0.012764,0.039245,0.021721,0.022306,0.034166,4
5,0.057384,0.147217,0.245109,0.027351,0.013350,0.015553,0.056138,0.102838,0.024279,0.013464,0.056257,0.002196,5
6,0.013793,0.018448,0.009191,0.037398,0.119724,0.028930,0.203289,0.081937,0.012140,0.030489,0.090822,0.045658,6
7,0.003549,0.025894,0.025674,0.008534,0.054980,0.012150,0.031600,0.099358,0.016798,0.055359,0.041375,0.021897,7
8,0.033216,0.020824,0.063552,0.069506,0.018354,0.051969,0.037526,0.021303,0.056038,0.193337,0.035950,0.126291,8
9,0.007175,0.017944,0.029150,0.023408,0.013991,0.075087,0.067100,0.039418,0.019434,0.082817,0.032879,0.066390,9
